# Data mining

In [1]:
import json
import csv
import tweepy
import re

In [2]:
def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'w',encoding="utf-8") as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended').items(100):
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])

In [3]:
consumer_key = 'x3yguSN30cfAefLBvBfi8VAh3'
consumer_secret = 'dNNZyD56IE6nXdxIcf6fgdFoDHQko0qK9iRHL9fLpdurG4Ejj6'
access_token = '1039440901792251904-5qy8WlPuJ2izG2uMpTS3zzlQ1P0H6Q'
access_token_secret = 'CULwd5uXcS2N3iNSG16VHFRzRcQisEefiAM1sqYdaCl2x'
    
hashtag_phrase = input('Hashtag Phrase ')

if __name__ == '__main__':
    search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)

Hashtag Phrase #flatearthers


# Text classification

In [1]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

In [1]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(text):
    return word_tokenize(text)

def remove_stop_words(text):
    stop_words= set(stopwords.words('english'))
    return([token.lower() for token in text if token not in stop_words])

def stem_words(text):
    stemmer= WordNetLemmatizer()
    return([stemmer.lemmatize(token) for token in text])

def stem_words_more(text):
    stemmer=PorterStemmer()
    return ([stemmer.stem(token) for token in text])

def remove_puncts(text):
    puncts= "~`!@#€$%^&*()_-+={[}]|\\:;'<,>.?/\''"
    return ([char for char in text if char not in puncts])

def clean_data(input_list):
    return_list=[]
    for li in input_list:
        return_list.append(stem_words(remove_puncts(remove_stop_words(tokenize(li)))))
    return return_list

def dummy(doc):
    return doc
  

In [4]:
df = pd.read_csv('C:/Users/jscul/Desktop/Advanced machine learning/flatearthers.csv')
df.head(10) 

,Tweeter,retweet count,liked count,language,Emotion,polarity,Postive/Neg,Sensitive,text,Unnamed: 9,Unnamed: 10
0,@bulkevans,0,0,en,anger,negative,-1,False,@MattBerch87 @dmarble1 Flat Earthers hate debates,NaN,NaN
1,@ShootaFairOne,0,0,en,unknown,positive,0,False,People want a reality show where flat earthers...,NaN,0.0
2,@politicians_sux,0,0,en,fear,negative,-1,False,@StitchASvGHeart @candybossinit @realDonaldTru...,NaN,NaN
3,@RedOphiuchus,0,0,en,fear,negative,-2,False,@MindfulCopper @VesuviaAdelia @insidehighered ...,NaN,NaN
4,@StitchASvGHeart,0,1,en,unknown,positive,-1,False,@politicians_sux @candybossinit @realDonaldTru...,NaN,NaN
5,@CoachDPenrod,0,0,en,unknown,negative,-2,False,Television show idea: a reality show where fla...,NaN,NaN
6,@keith_wentworth,0,0,en,joy,neutral,-2,False,@QuietlyRiot @_Gravity_Man @Itsjustanillus @MA...,NaN,NaN
7,@queso_the_rogue,0,0,en,unknown,negative,-3,False,@tsnurds Considering how many times flat earth...,NaN,NaN
8,@sars_cov,1,0,en,unknown,positive,-2,False,RT @cultofshinees: explain this flat earthers ...,NaN,NaN
9,@RealNoss,0,0,en,unknown,negative,-3,False,We need to load all the flat earthers onto a s...,NaN,NaN


In [2]:
# Check for NaN values:
df.isnull().sum()

Tweeter            0
retweet count      0
liked count        0
language           0
Emotion            0
polarity           0
Postive/Neg        0
Sensitive          0
text               0
Unnamed: 9       881
Unnamed: 10      885
dtype: int64

In [3]:
df['Emotion'].value_counts()

unknown     628
joy         168
sadness      28
anger        28
surprise     20
fear         12
disgust       2
Name: Emotion, dtype: int64

In [4]:
df['polarity'].value_counts()

positive    542
negative    248
neutral      96
Name: polarity, dtype: int64

# Emotion analysis

In [3]:
X = df['text']
y = df['Emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = clean_data(X_train)
X_test = clean_data(X_test)

print(X_train[0])

['rt', 'mattishellafat', 'bruh', 'earth', 'isnt', 'flat', 'foot', 'straight', 'i', 'mean', 'year', 'would', 'evolved', 'curved', 'fee�']


In [4]:
tfidf = TfidfVectorizer(analyzer='word',tokenizer=dummy,preprocessor=dummy,token_pattern=None)

X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

model= MultinomialNB(alpha=0.005)

model.fit(X_train,y_train)

MultinomialNB(alpha=0.005, class_prior=None, fit_prior=True)

In [5]:
# Form a prediction set
predictions = model.predict(X_test)

In [6]:
print(metrics.confusion_matrix(y_test,predictions))

[[  3   0   1   0   0   2]
 [  0   0   0   0   0   2]
 [  1   0  21   0   1  19]
 [  0   0   4   1   1   5]
 [  0   0   0   0   3   2]
 [  2   1   5   0   0 148]]


In [7]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

       anger       0.50      0.50      0.50         6
        fear       0.00      0.00      0.00         2
         joy       0.68      0.50      0.58        42
     sadness       1.00      0.09      0.17        11
    surprise       0.60      0.60      0.60         5
     unknown       0.83      0.95      0.89       156

    accuracy                           0.79       222
   macro avg       0.60      0.44      0.45       222
weighted avg       0.79      0.79      0.77       222



In [8]:
print(metrics.accuracy_score(y_test,predictions))

0.7927927927927928


# Polarity analysis

In [28]:
X = df['text']
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = clean_data(X_train)
X_test = clean_data(X_test)

print(X_train[0])

tfidf = TfidfVectorizer(analyzer='word',tokenizer=dummy,preprocessor=dummy,token_pattern=None)

X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

model= MultinomialNB(alpha=0.005)

model.fit(X_train,y_train)

predictions = model.predict(X_test)

['rt', 'mattishellafat', 'bruh', 'earth', 'isnt', 'flat', 'foot', 'straight', 'i', 'mean', 'year', 'would', 'evolved', 'curved', 'fee�']


In [29]:
print(metrics.confusion_matrix(y_test,predictions))

[[ 52   3  10]
 [  8   4   7]
 [ 12   8 118]]


In [30]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

    negative       0.72      0.80      0.76        65
     neutral       0.27      0.21      0.24        19
    positive       0.87      0.86      0.86       138

    accuracy                           0.78       222
   macro avg       0.62      0.62      0.62       222
weighted avg       0.78      0.78      0.78       222



# Gamestop crisis analysis

In [9]:
df1 = pd.read_csv('C:/Users/jscul/Desktop/Advanced machine learning/gamestop.csv',encoding='latin1')
df1.head(10)


,Tweeter,Polarity,Liked,retweet count,Language,sensitive,Text
0,@StonksDoctor,Positive,0,0,en,False,Hey @chamath I hear @GameStop and @ryancohen a...
1,@BillLam7718,Positive,0,0,en,False,@Silverchiquita Instead of spending your last ...
2,@Cito_da_gunman,Positive,1,0,en,False,Well I have my covid vaccine 1st shot appointm...
3,@shiftdeletenet,Positive,0,1,tr,False,Son zamanlarda ciddi bir de?er kayb? ya?ayan o...
4,@PyreSama,Neutral,0,0,en,False,@DereckSaurusRex And that's one of the many re...
5,@SwingVoterz,Positive,0,0,en,False,LMAO! This is awesome Michael Bolton - Break U...
6,@phendrome,Positive,1501,0,en,False,RT @ConanOBrien: Okay guys I finally caved and...
7,@JohnLothian,Positive,0,0,en,False,ESMA Chair Steven Maijoor Delivers Statement O...
8,@sudeep1110,Positive,0,0,en,False,Started using @withopal last week and my phone...
9,@ThaMaggle,Positive,148,0,en,False,RT @GamerSephiroth: is the gamestop in midgar ...


In [10]:
df1['Polarity'].value_counts()

Positive    669
Neutral     112
Negative     18
Name: Polarity, dtype: int64

In [11]:
X = df1['Text']
y = df1['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = clean_data(X_train)
X_test = clean_data(X_test)

print(X_train[0])

tfidf = TfidfVectorizer(analyzer='word',tokenizer=dummy,preprocessor=dummy,token_pattern=None)

X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

model= MultinomialNB(alpha=0.005)

model.fit(X_train,y_train)

predictions = model.predict(X_test)

['rt', 'appabend451', 'company', 'abuse', 'woman', 'that', "'s", 'horrible', 'support', 'woman', '``', 'okay', '``', 'tell', 'stop', '``', 'did', "n't", 'work', 'beg', 'sole', 'fema']


In [12]:
print(metrics.confusion_matrix(y_test,predictions))

[[  0   0   3]
 [  1  14  11]
 [  3   9 159]]


In [13]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         3
     Neutral       0.61      0.54      0.57        26
    Positive       0.92      0.93      0.92       171

    accuracy                           0.86       200
   macro avg       0.51      0.49      0.50       200
weighted avg       0.86      0.86      0.86       200



# Vader sentiment analysis

In [8]:
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jscul\AppData\Roaming\nltk_data...


True

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

df['polarity'].value_counts()
df.iloc[0][8]

sid.polarity_scores(df.iloc[0][8])

{'neg': 0.425, 'neu': 0.575, 'pos': 0.0, 'compound': -0.5719}

In [3]:
score_pos= []
score_neu= []
score_neg= []
for i in range(0,df.shape[0]):
    score = sid.polarity_scores(df.iloc[i][8])
    score1= score['pos']
    score_pos.append(score1)
    
    score2= score['neu']
    score_neu.append(score2)
    
    score3= score['neg']
    score_neg.append(score3)

In [4]:
df['positive_score']=score_pos
df['neutral_score']=score_neu
df['negative_score']=score_neg

In [5]:
df.head(5)

,Tweeter,retweet count,liked count,language,Emotion,polarity,Postive/Neg,Sensitive,text,Unnamed: 9,Unnamed: 10,positive_score,neutral_score,negative_score
0,@bulkevans,0,0,en,anger,negative,-1,False,@MattBerch87 @dmarble1 Flat Earthers hate debates,NaN,NaN,0.000,0.575,0.425
1,@ShootaFairOne,0,0,en,unknown,positive,0,False,People want a reality show where flat earthers...,NaN,0.0,0.085,0.915,0.000
2,@politicians_sux,0,0,en,fear,negative,-1,False,@StitchASvGHeart @candybossinit @realDonaldTru...,NaN,NaN,0.000,0.700,0.300
3,@RedOphiuchus,0,0,en,fear,negative,-2,False,@MindfulCopper @VesuviaAdelia @insidehighered ...,NaN,NaN,0.078,0.854,0.068
4,@StitchASvGHeart,0,1,en,unknown,positive,-1,False,@politicians_sux @candybossinit @realDonaldTru...,NaN,NaN,0.245,0.755,0.000


In [6]:
print('pos score: ',df['positive_score'].mean())
print('neu score: ',df['neutral_score'].mean())
print('neg score: ',df['negative_score'].mean())

pos score:  0.08506320541760727
neu score:  0.868602708803614
neg score:  0.046311512415349886


In [25]:
print('pos score: ',df['positive_score'].median())
print('neu score: ',df['neutral_score'].median())
print('neg score: ',df['negative_score'].median())

pos score:  0.0
neu score:  0.899
neg score:  0.0
